# Cargar y probar un modelo de Keras

## Cargar modelo desde .h5

El modelo se puede cargar desde un archivo .h5. Al ejecutar el siguiente cuadro de código podremos cargar uno desde el explorador de archivos.

In [61]:
from ipywidgets import FileUpload, Output
from tensorflow.keras import Model, models
import json
import os
from io import StringIO

if 'loaded_model' not in globals():
    global loaded_model
    loaded_model = None

def mostrar_h5_upload():
    h5_upload = FileUpload(accept= '.h5', 
    multiple=False)
    display(h5_upload)
    h5_upload.observe(handle_h5_upload, names='value')

def handle_h5_upload(change):
    *_, (_, f) = change['new'].items()
    h5_content = f['content']
    h5_uploaded_filepath = "kerasModelUpload.h5"
    with open(h5_uploaded_filepath,"w+b") as h5_file:
        h5_file.write(h5_content)
    global loaded_model
    loaded_model = models.load_model(h5_uploaded_filepath)
    print("Modelo cargado correctamente")
    print("Ejecuta el siguiente cuadro de código para verlo...")
    os.remove(h5_uploaded_filepath)
        
if(loaded_model is not None):
    answer = input("Ya hay un modelo cargado ¿Quieres borrarlo? [s/n]: ")
    if answer == "s":
        loaded_model = None
        print("Se ha borrado el modelo que se había cargado")
        mostrar_h5_upload()
    elif answer == "n":
        print("No se ha borrado el modelo.")
    else:
        print("No se ha borrado el modelo.")
else:
    loaded_model=None
    mostrar_h5_upload()

Ya hay un modelo cargado ¿Quieres borrarlo? [s/n]: s
Se ha borrado el modelo que se había cargado


FileUpload(value={}, accept='.h5', description='Upload')

In [55]:
loaded_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 126, 126, 4)       40        
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 42, 42, 4)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 38, 38, 8)         808       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 12, 12, 8)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 6, 6, 16)          6288      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 3, 3, 16)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 144)              

Tambien podemos cargar el modelo manualmente si sabemos el filepath del fichero .h5.

In [62]:
h5_filepath = "ivanNetConAugmentation-final.h5" # Sustituir por el filepath del archivo a cargar
loaded_model = models.load_model(h5_filepath)
loaded_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 124, 124, 20)      520       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 120, 120, 20)      10020     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 60, 60, 20)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 58, 58, 40)        7240      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 56, 56, 40)        14440     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 28, 28, 40)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 26, 26, 80)       

## Probando el modelo

### Cargar los datos

In [56]:
import os
# Desde google.colab: pip install wget
import wget
import zipfile
import sys

# Code from: https://stackoverflow.com/questions/58125279/python-wget-module-doesnt-show-progress-bar#answer-61346454        
def bar_progress(current, total, width=80):
  progress_message = "Downloading: %d%% [%d / %d] bytes" % (current / total * 100, current, total)
  # Don't use print() as it will print in new line every time.
  sys.stdout.write("\r" + progress_message)
  sys.stdout.flush()

dataset_zip_filename = "dataset.zip"
dataset_folder = dataset_zip_filename.replace(".zip", "")
dataset_zip_url = "https://zenodo.org/record/3902223/files/dataset-single-hand-finger-count.zip"
if(not os.path.isfile(dataset_zip_filename)):
    print(f"Se descargará el dataset desde '{dataset_zip_url}'")
    #command_wget = f"wget --no-check-certificate -O {dataset_zip_filename} {dataset_zip_url}"
    wget.download(dataset_zip_url,dataset_zip_filename, bar=bar_progress)
    #command_wget_res = os.system(command_wget)
    if(os.path.isfile(dataset_zip_filename) and not os.path.exists(dataset_folder)):
        print(f"Dataset descargado con éxito en '{os.getcwd()}/{dataset_folder}'")
        zip_ref = zipfile.ZipFile(dataset_zip_filename, 'r')
        zip_ref.extractall(dataset_folder)
        zip_ref.close()
        print(f"El dataset se ha descomprimido en '{os.getcwd()}/{dataset_folder}'")
    
else:
    print(f"El archivo '{dataset_zip_filename}' ya existe en la carpeta '{os.getcwd()}' así que no se ha descargado.")
    if(not os.path.exists(dataset_folder)):
        print(f"El archivo {dataset_zip_filename} no parece haber sido descomprimido, se procede a descomprimir.")
        zip_ref = zipfile.ZipFile(dataset_zip_filename, 'r')
        zip_ref.extractall(dataset_folder)
        zip_ref.close()
        print(f"El archivo {dataset_zip_filename} se ha descromprimido con éxito en '{os.getcwd()}/{dataset_folder}'")
    else:
        print(f"El dataset está descomprimido en '{os.getcwd()}/{dataset_folder}'")

El archivo 'dataset.zip' ya existe en la carpeta '/home/carlos/projects/keras-cnn' así que no se ha descargado.
El dataset está descomprimido en '/home/carlos/projects/keras-cnn/dataset'


In [41]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

simple_datagen = ImageDataGenerator(rescale = 1.0/255.)

dataset_folder="dataset"
batch_size=30

train_dir=f"{dataset_folder}/train"
test_dir=f"{dataset_folder}/test"
val_dir=f"{dataset_folder}/val"

train_set = simple_datagen.flow_from_directory(train_dir,
                  batch_size=batch_size,
                  class_mode='categorical',
                  target_size=(128, 128),
                  color_mode="grayscale")

test_set = simple_datagen.flow_from_directory(test_dir,
                  batch_size=batch_size,
                  class_mode = 'categorical',
                  target_size = (128, 128),
                  color_mode="grayscale")

val_set = simple_datagen.flow_from_directory(val_dir,
                  batch_size=batch_size,
                  class_mode = 'categorical',
                  target_size = (128, 128),
                  color_mode="grayscale")

Found 14400 images belonging to 6 classes.
Found 3600 images belonging to 6 classes.
Found 3600 images belonging to 6 classes.


### Evaluar

In [63]:
metrics_test = loaded_model.evaluate(
             test_set,
             batch_size=30,
             steps= test_set.n // 30,
             verbose=1,
             return_dict=True)

acc_test    = metrics_test['accuracy']
loss_test   = metrics_test['loss']

print(f"Precisión para el conjunto de Prueba: {acc_test}")
print(f"Pérdida para el conjunto de Prueba: {loss_test}")

120/120 [==============================] - 26s 216ms/step - loss: 0.0155 - accuracy: 0.9956
Precisión para el conjunto de Prueba: 0.995555579662323
Pérdida para el conjunto de Prueba: 0.015504955314099789


### Evaluar con data augmentation

In [51]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

augmented_datagen = ImageDataGenerator(rescale = 1.0/255.,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      vertical_flip=True,
                                      rotation_range=90)

train_dir=f"{dataset_folder}/train"
test_dir=f"{dataset_folder}/test"
val_dir=f"{dataset_folder}/val"

aug_train_set = augmented_datagen.flow_from_directory(train_dir,
                  batch_size=30,
                  class_mode='categorical',
                  target_size=(128, 128),
                  color_mode="grayscale")

aug_test_set = augmented_datagen.flow_from_directory(test_dir,
                  batch_size=30,
                  class_mode = 'categorical',
                  target_size = (128, 128),
                  color_mode="grayscale")

aug_val_set = augmented_datagen.flow_from_directory(val_dir,
                  batch_size=30,
                  class_mode = 'categorical',
                  target_size = (128, 128),
                  color_mode="grayscale")

batch_size = 30
aug_steps_per_epoch = aug_train_set.n // batch_size
aug_validation_steps = aug_val_set.n // batch_size
print("Pasos por época: " + str(aug_steps_per_epoch) + "    Pasos de validación: " + str(aug_validation_steps))

Found 14400 images belonging to 6 classes.
Found 3600 images belonging to 6 classes.
Found 3600 images belonging to 6 classes.
Pasos por época: 480    Pasos de validación: 120


In [58]:
metrics_test_aug2 = loaded_model.evaluate(
             aug_test_set,
             batch_size=30,
             steps= aug_test_set.n // batch_size,
             verbose=1,
             return_dict=True)

120/120 [==============================] - 6s 48ms/step - loss: 0.1299 - accuracy: 0.9567


In [59]:
metrics_test_aug2 = loaded_model.evaluate(
             aug_val_set,
             batch_size=30,
             steps= aug_test_set.n // batch_size,
             verbose=1,
             return_dict=True)

120/120 [==============================] - 6s 47ms/step - loss: 0.1380 - accuracy: 0.9533


In [60]:
aug_train_set = loaded_model.evaluate(
             aug_val_set,
             batch_size=30,
             steps= aug_test_set.n // batch_size,
             verbose=1,
             return_dict=True)

120/120 [==============================] - 6s 48ms/step - loss: 0.1161 - accuracy: 0.9578
